In [1]:
import pandas as pd

# Load the CSV file
csv_file_path = ".\Lyrics_csv\Radiohead_lyrics.csv"  # Replace with your CSV file path
df = pd.read_csv(csv_file_path)

# Combine all lyrics for pretraining, retaining line breaks
all_lyrics = df['Lyrics'].str.cat(sep='\n\n')

# Save all artist's lyrics to a text file with UTF-8 encoding
with open("artist_lyrics.txt", "w", encoding="utf-8") as file:
    file.write(all_lyrics)

# Extract and save specific song's lyrics
specific_song_title = "No Surprises"  # Replace with the specific song title
song_lyrics = df[df['Song'] == specific_song_title]['Lyrics'].values[0]
with open("song_lyrics.txt", "w", encoding="utf-8") as file:
    file.write(song_lyrics)


<>:4: SyntaxWarning: invalid escape sequence '\L'
<>:4: SyntaxWarning: invalid escape sequence '\L'
C:\Users\anshu\AppData\Local\Temp\ipykernel_17232\1932639874.py:4: SyntaxWarning: invalid escape sequence '\L'
  csv_file_path = ".\Lyrics_csv\Radiohead_lyrics.csv"  # Replace with your CSV file path


: 

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained Falcon model and tokenizer
model_name = "tiiuae/falcon-7b"  # Specify model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

artist_lyrics_path = "artist_lyrics.txt"
dataset = load_dataset(artist_lyrics_path, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./falcon-artist",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train the model
trainer.train()


In [ ]:
# Path to the specific song's lyrics dataset
song_lyrics_path = "song_lyrics.txt"
song_dataset = load_dataset(song_lyrics_path, tokenizer)

# Update training arguments for fine-tuning
fine_tuning_args = TrainingArguments(
    output_dir="./falcon-song",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer for fine-tuning
fine_tuner = Trainer(
    model=model,
    args=fine_tuning_args,
    data_collator=data_collator,
    train_dataset=song_dataset,
)

# Fine-tune the model
fine_tuner.train()
